### Lasso Regression

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/asheshlalshrestha/Desktop/UB/Research/CarePathways/Models/Datasets/preprocessed_dataset.csv")

In [3]:
df = df.drop(["Unnamed: 0","id"],axis=1)
df

,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,21,0.485152,1,0.003233,0.001623,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,-0.308119,0,-1.497964,-0.362476,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,-0.491182,0,0.003233,0.001623,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,1.705571,1,1.989114,-0.766671,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,1.522508,1,0.003233,0.001623,1.0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,2,-0.613224,0,-0.235617,-0.417097,0.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
502,13,0.485152,0,0.003233,-1.378426,0.0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
503,19,0.912299,1,-1.110511,-0.832216,0.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,9,0.485152,1,0.003233,0.079954,0.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Splitting X and y

In [4]:
from sklearn.model_selection import train_test_split
X = df.drop("hospital_length_of_stay",axis =1)
y = df['hospital_length_of_stay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (354, 745)
X_test shape: (152, 745)
y_train shape: (354,)
y_test shape: (152,)


In [5]:
X

,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),Diabetes,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,0.485152,1,0.003233,0.001623,0.0,0,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.308119,0,-1.497964,-0.362476,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.491182,0,0.003233,0.001623,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.705571,1,1.989114,-0.766671,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.522508,1,0.003233,0.001623,1.0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.613224,0,-0.235617,-0.417097,0.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
502,0.485152,0,0.003233,-1.378426,0.0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
503,0.912299,1,-1.110511,-0.832216,0.0,1,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,0.485152,1,0.003233,0.079954,0.0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [279]:
X_train

,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),Diabetes,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
5,-1.345475,1,1.264202,1.734969,0.0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,1.400466,1,0.003233,0.001623,0.0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,-0.796287,0,0.003233,-0.417097,0.0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1.278424,0,-0.110632,-0.378862,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
468,0.058006,1,1.014232,0.451376,0.0,0,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1.644550,1,0.003233,0.001623,0.0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,0.119027,1,0.003233,0.001623,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
348,1.095362,1,2.001612,0.571542,0.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
435,0.546173,1,0.089344,-0.837679,1.0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Implenting lasso regression to the original dataset

In [281]:
from sklearn.linear_model import Lasso

# Apply Lasso Regression
lasso = Lasso(alpha=0.1)  # Specify the regularization parameter alpha
lasso.fit(X_train, y_train)
df_importance = pd.DataFrame({'Feature': X.columns, 'Importance': lasso.coef_}).sort_values(by='Importance', ascending=False)
important_features = df_importance[df_importance['Importance']>0]
important_features.reset_index()

,index,Feature,Importance
0,496,Lansoprazole,28.768409
1,554,Isordil,18.525977
2,123,Polyethylene Glycol,8.727072
3,12,Kidney disease,4.337360
4,85,Amlodipine,4.228977
5,185,Avapro,3.333709
6,36,reason_for_admission_Fever of unknown origin [...,2.839970
7,211,Atacand,2.645734
8,227,Allopurinol,2.485787
9,24,Hematological,2.052317


### Important features are those which have importance of more than 0

In [282]:
# Print feature importances
important_features = df_importance[df_importance['Importance']>0]
print("Total important features:",len(important_features))
important_features

Total important features: 28


,Feature,Importance
496,Lansoprazole,28.768409
554,Isordil,18.525977
123,Polyethylene Glycol,8.727072
12,Kidney disease,4.337360
85,Amlodipine,4.228977
185,Avapro,3.333709
36,reason_for_admission_Fever of unknown origin [...,2.839970
211,Atacand,2.645734
227,Allopurinol,2.485787
24,Hematological,2.052317


### Dropping the unimportant features from X

In [283]:
feature_array = important_features['Feature'].values

cols_to_drop = [col for col in X.columns if col not in feature_array]

new_x_for_lasso = X.drop(cols_to_drop,axis=1)
new_x_for_lasso

,age,sex,Hypertension,Kidney disease,Dementia [any etiology],Stroke,Chronic renal,Hematological,reason_for_admission_Fever of unknown origin [R50.9],reason_for_admission_Pneumonia [J18.9],...,Avapro,Aspirin Tablets,Ezetrol,Atacand,Allopurinol,Cipralex,Calcitriol,Tylenol W Codeine No Tab,Lansoprazole,Isordil
0,0.485152,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.308119,0,1,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.491182,0,1,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.705571,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.522508,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.613224,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,0.485152,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,0.912299,1,1,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,0.485152,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Using the important features in the model 

In [284]:
X_train, X_test, y_train, y_test = train_test_split(new_x_for_lasso, y, test_size=0.3, random_state=42)
lasso.fit(X_train, y_train)
y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)
mse_y_test = mean_squared_error(y_test, y_pred_test)
mse_y_train = mean_squared_error(y_train, y_pred_train)
print("Mean Squared Error train:", mse_y_train)
print("Mean Squared Error test:", mse_y_test)

Mean Squared Error train: 95.95209765992696
Mean Squared Error test: 179.8218392640842


In [285]:
df_train = pd.DataFrame({'y_train': y_train, 'y_pred_train': y_pred_train})

# Create a scatter plot using Plotly Express
fig = px.scatter(df_train, x='y_train', y='y_pred_train', title='y_train vs y_pred_train')
fig.update_layout(xaxis_title='y_train', yaxis_title='y_pred_train')
fig.show()

df_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test})

# Create a scatter plot using Plotly Express
fig = px.scatter(df_test, x='y_test', y='y_pred_test', color_discrete_sequence=['green'], title='y_test vs y_pred_test')
fig.update_layout(xaxis_title='y_test', yaxis_title='y_pred_test')
fig.show()

### Working on the final df to show the predicted and the actual values

In [286]:
## final_df is the df consisting of the id and the original data values
final_df = pd.read_csv('Datasets/new.csv')
final_df = final_df.drop(["Unnamed: 0"],axis=1)

In [287]:
final_df.head()

,id,hospital_length_of_stay,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,...,Respiractin,Immunity Advance,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil
0,1,21,74,1,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,5,61,0,154.900000,73.600000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,7,58,0,166.911029,80.265922,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,9,94,1,182.800000,66.200000,0.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,9,91,1,166.911029,80.265922,1.0,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [288]:
## we already have dataframe named df_test and df_train created above to plot the scatter plot

## we will rename the columns initially
df_test.rename(columns={'y_test': 'HLOS','y_pred_test':'predicted'}, inplace=True)
df_train.rename(columns={'y_train': 'HLOS','y_pred_train':'predicted'}, inplace=True)

## concat the two dataframes
combined_df = pd.concat([df_test, df_train])

## finally merge the combined_df based on index with the with the final_df
final_df = pd.merge(final_df, combined_df, left_index=True, right_index=True, suffixes=('', '_combined'))

## dropping the column hospital_length_of_stay to avoid confusion as HOLS has already been added 
final_df = final_df.drop('hospital_length_of_stay',axis = 1)
final_df.head()

,id,age,sex,height,weight,smoking_history,previous_er_visit_within_14_days,admission_disposition,Hypertension,Chronic cardiac disease (not hypertension),...,Vesicare,Zaxine,Quinine Sulfate,Desvenlafaxine,Glucosamine,Turmeric,Cogentin Tab,Elavil,HLOS,predicted
0,1,74,1,166.911029,80.265922,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,12.144272
1,2,61,0,154.900000,73.600000,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,11.204107
2,3,58,0,166.911029,80.265922,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,10.916416
3,4,94,1,182.800000,66.200000,0.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,14.062212
4,5,91,1,166.911029,80.265922,1.0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,13.774521


In [255]:
# final_df.to_csv("Datasets/df_with_predicted_values")